In [1]:
#!pip install autokeras
import autokeras as ak
import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, Accuracy, AUC
from metrics import dice_coef, dice_loss, iou
import tensorboard
import os

In [2]:
# load the augmented data from the local data directory:

train_dir = "Data/Processed_Data/train"
test_dir = "Data/Processed_Data/test"
valid_dir = "Data/Processed_Data/valid"

#batch_size = 8
img_height = 64
img_width = 64

train_data = ak.image_dataset_from_directory(
    train_dir,
    # Set seed to ensure the same split when loading testing data.
    seed=123,
    image_size=(img_height, img_width),
    color_mode='rgb',
    validation_split=0.95, subset='training',
)

test_data = ak.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(img_height, img_width),
    color_mode='rgb',
)

valid_data = ak.image_dataset_from_directory(
    valid_dir,
    seed=123,
    image_size=(img_height, img_width),
    color_mode='rgb',
)

Found 72096 files belonging to 8 classes.
Using 3605 files for training.
Found 3799 files belonging to 8 classes.
Found 3796 files belonging to 8 classes.


In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# create a directory to save the files
create_dir("files")

In [5]:
# Create a list for callbacks

# ModelCheckpoint saves the weight file while training
# ReduceLROnPlateau reduces the learning rate by the given factor
# CSVLogger to keep a log of all metrics while training
# TensorBoard for visualizations
# Earlystopping is used to reduce the overfitting (when val_loss matrix stops improving for continuously 20 epocs,
# # it stops the training)
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard

# When the model runs, we need a path to save the weight files:
model_path = "files\model.h5"

# The below csv path will be used to save all the metrices values while evaluation
csv_path = "files\data.csv"


callbacks = [
    ModelCheckpoint(model_path, verbose=0, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience=5, min_lr=1e-7, verbose =1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False )]

# Image Classifier Model

In [10]:
# Lets try an image classifier for the same dataset:
IC_model = ak.ImageClassifier(max_trials=15, overwrite=False, seed=123)

INFO:tensorflow:Reloading Oracle from existing project .\image_classifier\oracle.json
INFO:tensorflow:Reloading Tuner from .\image_classifier\tuner0.json


In [11]:
IC_model.fit(train_data, epochs=30, validation_split=0.2)


Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
image_block_1/n...|True              |True              
image_block_1/a...|True              |True              
image_block_1/b...|efficient         |efficient         
classification_...|global_avg        |global_avg        
classification_...|0                 |0                 
optimizer         |adam              |adam              
learning_rate     |2e-05             |2e-05             
image_block_1/i...|0.1               |0.1               
image_block_1/i...|True              |True              
image_block_1/i...|False             |False             
image_block_1/i...|0                 |0                 
image_block_1/i...|0                 |0                 
image_block_1/i...|0                 |0                 
image_block_1/e...|True              |True              
image_block_1/e...|True              |True              
image_block_1/e...|b7                |b7                
imag

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[1,56,56,288] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model/efficientnetb7/block2f_activation/mul_1-0-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_390916]

In [12]:
# export the created model:
best_IC_model = IC_model.export_model()
best_IC_model.save("best_model.h5")

In [ ]:
val_predictions = IC_model.predict(valid_data, batch_size=16, verbose=1)

In [13]:
model = tf.keras.models.load_model("best_model.h5")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 64, 64, 3)        7         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0     

# Auto Model

In [ ]:
# lets configure a trial model:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ConvBlock()(output_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

In [ ]:
#auto_model = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=2, objective='val_accuracy', seed=123)

In [ ]:
# just running the model on the test data to check the performance
proposed_model = auto_model.fit(test_data, epochs=10, verbose =2, callbacks=callbacks, validation_split=0.2)

In [ ]:
# export the created model:
best_model = auto_model.export_model()
best_model.save("bestmodel_ak.h5")

In [ ]:
auto_model.predict(test_data, batch_size=16, verbose=1)

In [ ]:
model = tf.keras.models.load_model("bestmodel_ak.h5")
model.summary()